In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('Data_Private/prepared/cleaned_VacMe_Gemeinden_ueber_500.csv')

In [3]:
data.head()

,Registration,Geschlecht,Vollstaendiger_Impfschutz,Imfgruppe,Wohngemeinde,Erstimpfung ODI Typ,Erstimpfung Gemeinde,Erstimpfung,Zweitimpfung ODI Typ,Zweitimpfung Gemeinde,Zweitimpfung
0,2021-02-28,MAENNLICH,WAHR,N,Thun,Impfzentrum,Thun,02.06.2021,Impfzentrum,Thun,30.06.2021
1,2021-10-02,WEIBLICH,WAHR,G,Thun,Impfzentrum,Thun,03.07.2021,Impfzentrum,Thun,02.08.2021
2,2021-07-05,WEIBLICH,FALSCH,F,Thun,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-10-03,MAENNLICH,WAHR,N,Thun,Impfzentrum,Thun,03.08.2021,Impfzentrum,Thun,31.08.2021
4,2021-01-15,WEIBLICH,WAHR,A,Thun,Impfzentrum,Bern,11.02.2021,Impfzentrum,Bern,11.03.2021


In [4]:
data['Erstimpfung ODI Typ'].unique()

array(['Impfzentrum', nan, 'Spital', 'Arztpraxis', 'Mobiles Impfteam',
       'Apotheke', 'Alters- und Pflegeheim', 'Impfpraxis', 'Betriebe',
       'Andere', 'Test'], dtype=object)

In [5]:
gemeinden = data['Wohngemeinde'].unique()
gemeinden = [g for g in gemeinden if type(g) is str]

In [6]:
geimpfte_thuner = data[data['Wohngemeinde']=='Thun'].shape[0]

In [7]:
geimpfte = data[data['Vollstaendiger_Impfschutz'] == 'WAHR']

In [8]:
geimpfte.shape[0]

721677

In [9]:
geimpfte[geimpfte['Zweitimpfung Gemeinde'] != geimpfte['Erstimpfung Gemeinde']].shape[0]

39553

In [10]:
anzahl_heimgeimpfte = geimpfte[geimpfte['Wohngemeinde'] == geimpfte['Erstimpfung Gemeinde']].shape[0]

In [11]:
anzahl_geimpfte = geimpfte.shape[0]

In [12]:
print('total score: ', anzahl_heimgeimpfte / anzahl_geimpfte)

total score:  0.252872129775509


In [13]:
_eigenimpfscore_map = {}
for g in sorted(gemeinden):
    einw = geimpfte[geimpfte['Wohngemeinde']==g]
    heim_impfungen = einw[g == einw['Erstimpfung Gemeinde']].shape[0]
    anz_einw = einw.shape[0]
    score = heim_impfungen / anz_einw
    _eigenimpfscore_map[g] = score

In [14]:
_hatImpzentrum_map = {}
for g in sorted(gemeinden):
    impfz = geimpfte[(geimpfte['Erstimpfung ODI Typ'] == 'Impfzentrum') & (geimpfte['Erstimpfung Gemeinde'] == g)]
    if impfz.shape[0] > 0:
        _hatImpzentrum_map[g] = True
    else:
        _hatImpzentrum_map[g] = False
print(_hatImpzentrum_map)

{'Aarberg': False, 'Aarwangen': False, 'Adelboden': False, 'Aefligen': False, 'Aegerten': False, 'Aeschi bei Spiez': False, 'Affoltern im Emmental': False, 'Alchenstorf': False, 'Allmendingen': False, 'Amsoldingen': False, 'Arch': False, 'Arni (BE)': False, 'Attiswil': False, 'Bannwil': False, 'Bargen (BE)': False, 'Beatenberg': False, 'Bellmund': False, 'Belp': False, 'Bern': True, 'Bettenhausen': False, 'Biel/Bienne': True, 'Biglen': False, 'Bleienbach': False, 'Blumenstein': False, 'Bolligen': False, 'Boltigen': False, 'Bowil': False, 'Bremgarten bei Bern': False, 'Brienz (BE)': False, 'Brügg': False, 'Brüttelen': False, 'Buchholterberg': False, 'Burgdorf': True, 'Burgistein': False, 'Bäriswil': False, 'Bätterkinden': False, 'Bönigen': False, 'Büetigen': False, 'Büren an der Aare': False, 'Corgémont': False, 'Cortébert': False, 'Court': False, 'Courtelary': False, 'Diemtigen': False, 'Diessbach bei Büren': False, 'Dotzigen': False, 'Därstetten': False, 'Dürrenroth': False, 'Eggiwil'

In [15]:
df = pd.DataFrame({'score': _eigenimpfscore_map, 'hatImpfzentrum': _hatImpzentrum_map})

In [16]:
df.sort_values('score')

,score,hatImpfzentrum
Mattstetten,0.000000,False
Rapperswil (BE),0.000000,False
Jens,0.000000,False
Reutigen,0.000000,False
Rohrbach,0.000000,False
...,...,...
Tavannes,0.610364,True
Adelboden,0.732905,False
Thun,0.753158,True
Bern,0.819131,True


In [17]:
# df.to_csv('Data_Public/eigenimpfscore.csv')

In [18]:
def clean_gemeindecode():
    portrait = pd.read_csv('Data_Public/generated/regionalportraets2021Gemeinden.csv', sep=";",
                 encoding='Latin1', header=5,
                 parse_dates=True )
    codes = portrait[['Gemeindecode', 'Gemeindename']].dropna()
    return codes.set_index('Gemeindecode')

In [19]:
codes = clean_gemeindecode()

In [20]:
codes.head()

,Gemeindename
Gemeindecode,
1,Aeugst am Albis
2,Affoltern am Albis
3,Bonstetten
4,Hausen am Albis
5,Hedingen


In [27]:
merged = pd.merge(codes, df, left_on='Gemeindename', right_index=True).reset_index()
merged.head()

,Gemeindecode,Gemeindename,score,hatImpfzentrum
0,301,Aarberg,0.308426,False
1,302,Bargen (BE),0.000000,False
2,303,Grossaffoltern,0.000000,False
3,304,Kallnach,0.039074,False
4,305,Kappelen,0.000000,False


In [42]:
merged = merged.rename(columns={'Gemeindecode': 'bfs_nummer', 'score': 'value'})

In [43]:
eigenscore = merged[['bfs_nummer', 'value']]

In [46]:
eigenscore.to_json(orient='records')

'[{"bfs_nummer":"301","value":0.3084255843},{"bfs_nummer":"302","value":0.0},{"bfs_nummer":"303","value":0.0},{"bfs_nummer":"304","value":0.0390738061},{"bfs_nummer":"305","value":0.0},{"bfs_nummer":"306","value":0.3148584906},{"bfs_nummer":"307","value":0.0484554815},{"bfs_nummer":"309","value":0.0},{"bfs_nummer":"310","value":0.0},{"bfs_nummer":"311","value":0.0592320261},{"bfs_nummer":"312","value":0.1476386978},{"bfs_nummer":"321","value":0.1687734882},{"bfs_nummer":"323","value":0.0},{"bfs_nummer":"324","value":0.0},{"bfs_nummer":"326","value":0.0},{"bfs_nummer":"329","value":0.8709039842},{"bfs_nummer":"331","value":0.0146878825},{"bfs_nummer":"332","value":0.0},{"bfs_nummer":"333","value":0.1530886303},{"bfs_nummer":"337","value":0.113961312},{"bfs_nummer":"338","value":0.0},{"bfs_nummer":"340","value":0.0},{"bfs_nummer":"341","value":0.0},{"bfs_nummer":"342","value":0.1070184171},{"bfs_nummer":"344","value":0.0},{"bfs_nummer":"345","value":0.0},{"bfs_nummer":"351","value":0.819

In [40]:
eigenscore['slider_value'] = 'Total'

/tmp/ipykernel_43407/1411881114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eigenscore['slider_value'] = 'Total'


In [41]:
eigenscore.to_csv('')

,bfs_nummer,value,slider_value
0,301,0.308426,Total
1,302,0.000000,Total
2,303,0.000000,Total
3,304,0.039074,Total
4,305,0.000000,Total
...,...,...,...
260,988,0.004454,Total
261,989,0.000000,Total
262,991,0.000000,Total
263,992,0.000767,Total
